In [1]:
!pip install -q "transformers>=4.36.0" accelerate bitsandbytes sentencepiece sentence-transformers

In [2]:
# Loading Mistral Model

from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util
import torch, json, time, re

LLM_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# Tokenizer
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_NAME)

if llm_tokenizer.pad_token is None:
    llm_tokenizer.pad_token = llm_tokenizer.eos_token
llm_tokenizer.padding_side = "right"

llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_NAME,
    load_in_4bit=True,          # <- key part
    device_map="auto",          # let HF spread across GPU/CPU if needed
)

# SentenceTransformer on GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
verb_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

#### In chat style prompting:
1. Role type- "system", Sets the model’s behavior, identity, and global rules
2. Role type- "user",  Represents what the end-user is asking
3. Role type- "assistant", Represents the model’s own prior outputs

In [67]:
import math

In [68]:
# Few-shot prompting
base_messages = [
    {
        "role": "system",
        "content": (
            "You are an expert text parser for a text-based RPG game. "
            "Your job is to extract three fields from the player's command: "
            "\"verb\", \"object\", and \"instrument\". "
            "Return ONLY a valid JSON object with exactly these keys and no extra text, "
            "no notes, no explanations, no trailing text."
        )
    },
    # --- verb + object + instrument ---
    {
        "role": "user",
        "content": "Attack Gascoigne with Pistol"
    },
    {
        "role": "assistant",
        "content": '{"verb": "attack", "object": "Gascoigne", "instrument": "Pistol"}'
    },
    {
        "role": "user",
        "content": "Shoot the Snatcher in Yahar\'gul with the Pistol"
    },
    {
        "role": "assistant",
        "content": '{"verb": "shoot", "object": "Snatcher", "instrument": "Pistol"}'
    },
    {
        "role": "user",
        "content": "Slash the Rabid Dog using my Saw Cleaver"
    },
    {
        "role": "assistant",
        "content": '{"verb": "slash", "object": "Rabid Dog", "instrument": "Saw Cleaver"}'
    },

    # --- verb + object ---
    {
        "role": "user",
        "content": "Talk to Gilbert"
    },
    {
        "role": "assistant",
        "content": '{"verb": "talk", "object": "Gilbert", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Use a Blood Vial"
    },
    {
        "role": "assistant",
        "content": '{"verb": "use", "object": "Blood Vial", "instrument": ""}'
    },
    {
        "role": "user",
        "content": "Play the Music Box"
    },
    {
        "role": "assistant",
        "content": '{"verb": "play", "object": "Music Box", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Travel to Cathedral Ward"
    },
    {
        "role": "assistant",
        "content": '{"verb": "travel", "object": "Cathedral Ward", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Teleport to 4"
    },
    {
        "role": "assistant",
        "content": '{"verb": "teleport", "object": "4", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Who is Gilbert"
    },
    {
        "role": "assistant",
        "content": '{"verb": "Who", "object": "Gilbert", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Pick up the Molotov Cocktail"
    },
    {
        "role": "assistant",
        "content": '{"verb": "pick up", "object": "Molotov Cocktail", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Inspect the Huntsman"
    },
    {
        "role": "assistant",
        "content": '{"verb": "inspect", "object": "Huntsman", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Inspect the Handwritten Note"
    },
    {
        "role": "assistant",
        "content": '{"verb": "inspect", "object": "Handwritten Note", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Read the note in Central Yharnam"
    },
    {
        "role": "assistant",
        "content": '{"verb": "read", "object": "note", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Upgrade my Saw Cleaver"
    },
    {
        "role": "assistant",
        "content": '{"verb": "upgrade", "object": "Saw Cleaver", "instrument": ""}'
    },

    # --- Only verb examples ---
    {
        "role": "user",
        "content": "Heal"
    },
    {
        "role": "assistant",
        "content": '{"verb": "heal", "object": "", "instrument": ""}'
    },
    {
        "role": "user",
        "content": "restore"
    },
    {
        "role": "assistant",
        "content": '{"verb": "restore", "object": "", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "Status"
    },
    {
        "role": "assistant",
        "content": '{"verb": "status", "object": "", "instrument": ""}'
    },

    {
        "role": "user",
        "content": "inventory"
    },
    {
        "role": "assistant",
        "content": '{"verb": "inventory", "object": "", "instrument": ""}'
    },
    {
        "role": "user",
        "content": "Room"
    },
    {
        "role": "assistant",
        "content": '{"verb": "room", "object": "", "instrument": ""}'
    },
    {
        "role": "user",
        "content": "Reset"
    },
    {
        "role": "assistant",
        "content": '{"verb": "reset", "object": "", "instrument": ""}'
    },
]

# LLM-based command parser

def parse_command(user_text: str):
    messages = base_messages + [
        {"role": "user", "content": user_text}
    ]

    chat_text = llm_tokenizer.apply_chat_template(
        messages,
        tokenize=False
    )

    encoded = llm_tokenizer(
        chat_text,
        return_tensors="pt",
        padding=True
    )

    input_ids = encoded["input_ids"].to(llm_model.device)
    attention_mask = encoded["attention_mask"].to(llm_model.device)

    start = time.time()
    with torch.no_grad():
        outputs = llm_model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=32,
            do_sample=False,
            pad_token_id=llm_tokenizer.eos_token_id,  # suppresses the warning

        )

    gen_only = outputs[:, input_ids.shape[-1]:]
    raw = llm_tokenizer.decode(gen_only[0], skip_special_tokens=True).strip()

    m = re.search(r"\{.*\}", raw, flags=re.DOTALL)
    if not m:
        raise ValueError(f"LLM output did not contain JSON: {raw!r}")

    json_str = m.group(0).strip()

    try:
        parsed = json.loads(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Failed to parse JSON from model output: {raw!r}") from e

    return parsed

# Canon Actions Mapping

canonical_synonyms = {
    "attack": [
        "attack", "hit", "strike", "stab", "slash", "shoot", "bash", "punch"
    ],
    "talk": [
        "talk", "speak", "chat", "communicate", "converse"
    ],
    "use": [
        "use", "activate", "consume", "play", "light", "drink"
    ],
    "upgrade_weapon": [
        "upgrade", "fortify", "enhance", "strengthen", "improve", "sharpen weapon"
    ],
    "heal": [
        "heal", "restore", "recover", "recover hp", "regenerate"
    ],
    "travel": [
        "travel", "go", "walk", "move", "run", "head to"
    ],
    "get": [
        "get", "pick up", "pickup", "take", "grab", "collect", "loot", "equip"
    ],
    "inspect": [
        "inspect", "look", "examine", "check", "look at", "read", "observe"
    ],
    "inventory_stats": [
        "inventory", "open inventory", "show inventory", "inventory stats", "bag"
    ],
    "player_stats": [
        "player stats", "stats", "status", "show stats", "character sheet"
    ],
    "increase_health": [
        "increase health", "boost health", "raise health cap", "increase max hp"
    ],
    "describe_room": [
        "describe room", "look around", "look around the room",
        "describe surroundings"
    ],
    "teleport": [
        "teleport", "warp", "fast travel", "move to lamp"
    ],
    "reset_game": [
        "reset", "restart", "start over", "new game"
    ],
}

synonym_phrases = []
phrase_to_action = []

for action, phrases in canonical_synonyms.items():
    for p in phrases:
        synonym_phrases.append(p)
        phrase_to_action.append(action)

syn_embs = verb_model.encode(synonym_phrases, convert_to_tensor=True)


def get_action(raw_verb: str, obj: str = "", threshold: float = 0.25):
    """
    Map a raw verb (and sometimes object) to a canonical action.
    """
    verb = raw_verb.strip().lower()
    obj_l = obj.strip().lower()

    # added hard rules so model doesn't get confused

    # Inventory: "show inventory", "open inventory", etc.
    if verb in ["show", "open", "display"] and "inventory" in obj_l:
        return "inventory_stats", 1.0

    # Player stats: "show status", "status", etc.
    if verb in ["show", "display", "check"] and obj_l in ["status", "stats", "player", "my stats"]:
        return "player_stats", 1.0

    if verb in ["show", "display", "check"] and obj_l in ["room", "location", "place"]:
        return "describe_room", 1.0

    # Use lamp → teleport
    if verb in ["use", "light", "activate", "consume", "drink"] and "lamp" in obj_l:
        return "use", 1.0


    # 5) Fallback to verb-only embedding match
    text = verb
    emb = verb_model.encode(text, convert_to_tensor=True)
    scores = util.cos_sim(emb, syn_embs)[0]
    best_idx = int(torch.argmax(scores))
    best_score = float(scores[best_idx])

    if best_score < threshold:
        return None, best_score

    canonical = phrase_to_action[best_idx]
    return canonical, best_score

In [69]:
def run_command(user_text: str):
    """
    Full pipeline:
      1. LLM parses user_text into {verb, object, instrument}
      2. Embedding model maps verb+object → canonical action
      3. Dispatcher calls the corresponding Python function
    """

    #  Question shortcuts → INSPECT
    question_prefixes = [
        "who is ",
        "what is ",
        "what's ",
        "whats"
        "tell me about ",
    ]

    for prefix in question_prefixes:
        if user_text.startswith(prefix):
            # Extract everything after the prefix as the object
            obj_raw = user_text[len(prefix):].strip(" ?!.").strip()
            if not obj_raw:
                print("[ERROR] No object found in question.")
                return

            Inspect(obj_raw)
            return

    #  Parse
    parsed = parse_command(user_text)
    raw_verb = parsed.get("verb", "")
    obj = parsed.get("object", "")
    inst = parsed.get("instrument", "")


    # Map (verb, object) → canonical action
    action, score = get_action(raw_verb, obj)
    if action is None:
        print(f"[ERROR] Could not map verb+object {raw_verb!r} {obj!r} to a known action (score={score:.3f})")
        return


    # Dispatch
    func = ACTION_DISPATCH.get(action)
    if func is None:
        print(f"[ERROR] No handler for action {action!r}")
        return

    if action == "attack":
        func(obj, inst)
    elif action in ["use", "upgrade_weapon", "travel", "teleport", "get", "inspect", "talk"]:
        func(obj)
    else:
        func()


In [91]:
# %%
# Force remove old data and pull fresh from GitHub
import shutil
from pathlib import Path

# Remove old clone if it exists
if Path("/content/CMSC-491-Project").exists():
    shutil.rmtree("/content/CMSC-491-Project")
    print("🗑️  Removed old game data")

# Now clone fresh
!git clone -b main https://github.com/Khushika1805/CMSC-491-Project.git
print("✓ Cloned fresh game data from GitHub")

🗑️  Removed old game data
Cloning into 'CMSC-491-Project'...
remote: Enumerating objects: 276, done.
remote: Counting objects: 100% (276/276), done.
remote: Compressing objects: 100% (258/258), done.
remote: Total 276 (delta 60), reused 178 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (276/276), 107.31 KiB | 8.94 MiB/s, done.
Resolving deltas: 100% (60/60), done.
✓ Cloned fresh game data from GitHub


In [92]:
import json, copy

ROOT = Path("/content/CMSC-491-Project/Data")

def load_json_tree(root: Path):
    data = {}
    for path in root.rglob("*.json"):
        # e.g. Characters/gilbert.json → "Characters/gilbert"
        rel = path.relative_to(root).with_suffix("")
        key = "/".join(rel.parts)
        with path.open() as f:
            data[key] = json.load(f)
    return data

GAME_DATA_ORIG = load_json_tree(ROOT)
print("Loaded keys:", list(GAME_DATA_ORIG.keys())[:10])  # peek


Loaded keys: ['player_state', 'Locations/bay_of_pigs', 'Locations/upper_plaza', 'Locations/central_square', 'Locations/grand_cathedral', 'Locations/advent_plaza', 'Locations/stone_graveyard', 'Locations/catacomb_terrace', 'Locations/hosts_maze', 'Locations/crypt_of_eyes']


In [93]:
# Creating a copy s
GAME_DATA_SESSION1 = copy.deepcopy(GAME_DATA_ORIG)  # session copy

# Example edits during play:
GAME_DATA_SESSION1["Characters/gilbert"]["hp"] = 30
GAME_DATA_SESSION1["Items/blood_vial"]["quantity"] = 2

PLAYER_STATE_TEMPLATE = GAME_DATA_ORIG["player_state"]

# Per play-session:
PLAYER_STATE = copy.deepcopy(PLAYER_STATE_TEMPLATE)


In [94]:
# -- Helper Functions for Travel & Teleport --
LOCATION_INDEX = {}
for key, data in GAME_DATA_ORIG.items():
    if key.startswith("Locations/") and isinstance(data, dict) and "id" in data:
        LOCATION_INDEX[data["id"]] = data

def get_location_by_id(loc_id: str):
    return LOCATION_INDEX.get(loc_id)

def get_location_by_name(name: str):
    name = name.strip().lower()
    if name.startswith("to "):
        name = name[3:].strip()

    for loc in LOCATION_INDEX.values():
        loc_name = loc.get("name", "").lower()
        if loc_name == name:
            return loc

def location_has_lamp(loc_data) -> bool:
    # If we got a string, treat it as a location id and resolve it
    if isinstance(loc_data, str):
        loc_data = get_location_by_id(loc_data)
        if not loc_data:
            return False  # unknown location → no lamp

    for item in loc_data.get("items", []):
        if item.get("id") == "lamp" and item.get("count", 0) > 0:
            return True
    return False

## Implementing Action Logic

## ATTACK ACTION

In [95]:
from typing import List, Tuple
## HELPER FUNCTIONS FOR ATTACK ACTION

# Normalize text for case-insensitive matching.
def _normalize(text: str) -> str:
    return text.lower().strip()


# Resolve an item identifier from id/name/alias strings.
def _resolve_item_id(text):
    if not text:
        return None

    # Normalize input: lowercase and handle spaces/underscores/apostrophes
    normalized = text.strip().lower()

    # Create multiple variations to try
    variations = [
        normalized,
        normalized.replace(" ", "_"),
        normalized.replace("'", ""),
        normalized.replace("'", ""),  # smart apostrophe
        normalized.replace(" ", "_").replace("'", ""),
        normalized.replace(" ", "_").replace("'", ""),
        normalized.replace("'s", "s"),
        normalized.replace(" ", "_").replace("'s", "s"),
    ]

    # Try to find matching item
    for key in GAME_DATA_ORIG.keys():
        if not key.startswith("Items/"):
            continue

        item_id = key.replace("Items/", "")
        item_data = GAME_DATA_ORIG[key]
        item_name = item_data.get("name", "").lower()

        # Normalize item_id and item_name for comparison
        item_id_normalized = item_id.lower()
        item_name_normalized = item_name.replace("'", "").replace("'", "")

        # Check all variations
        for variant in variations:
            if (item_id_normalized == variant or
                item_name_normalized.replace(" ", "_") == variant or
                item_name_normalized == variant.replace("_", " ")):
                return item_id

    return None


# Resolve a character identifier from id/name/alias strings.
def _resolve_character_id(name: str) -> str | None:
    needle = _normalize(name)
    for key, payload in GAME_DATA_ORIG.items():
        if not key.startswith("Characters/"):
            continue
        if any(
            _normalize(str(val)) == needle
            for val in (
                payload.get("id"),
                payload.get("name"),
                payload.get("alias"),
            )
        ):
            return payload.get("id")
    return None


# Return the current in-memory player state.
def _load_player_state() -> dict:
    return GAME_DATA_ORIG.get("player_state", {})


# Return the current in-memory location payload.
def _load_location(loc_id: str) -> dict:
    return GAME_DATA_ORIG.get(f"Locations/{loc_id}", {})


# Update the in-memory player state.
def _save_player_state(state: dict):
    GAME_DATA_ORIG["player_state"] = state


# Update the in-memory location payload.
def _save_location(loc_id: str, payload: dict):
    GAME_DATA_ORIG[f"Locations/{loc_id}"] = payload


# Initialize or refresh enemy state (health, vulnerability) for this location.
def _init_enemy_state(
    player_state: dict,
    loc_id: str,
    char_id: str,
    count: int,
    base_health: int,
    base_vulnerable: bool,
) -> Tuple[List[int], bool]:
    """Returns (health_list, vulnerable_flag) for the enemy in this location."""
    char_state = player_state.setdefault("character_state", {})
    loc_state = char_state.setdefault(loc_id, {})
    entry = loc_state.setdefault(
        char_id, {"health": [base_health] * count, "vulnerable": base_vulnerable}
    )

    health_list: List[int] = list(entry.get("health", []))
    if len(health_list) < count:
        health_list.extend([base_health] * (count - len(health_list)))
    elif len(health_list) > count:
        health_list = health_list[:count]

    vulnerable = bool(entry.get("vulnerable", base_vulnerable))
    loc_state[char_id] = {"health": health_list, "vulnerable": vulnerable}
    return health_list, vulnerable


# Update stored enemy state, pruning if all are dead.
def _update_enemy_state(
    player_state: dict, loc_id: str, char_id: str, health_list: List[int], vulnerable: bool
):
    char_state = player_state.setdefault("character_state", {})
    loc_state = char_state.setdefault(loc_id, {})
    if all(hp <= 0 for hp in health_list):
        loc_state.pop(char_id, None)
    else:
        loc_state[char_id] = {"health": health_list, "vulnerable": vulnerable}


# Collect drops into player state and return human-readable summaries.
def _collect_drops(player_state: dict, drops: List[dict]) -> List[str]:
    """Apply dropped items to player state and return human-readable descriptions."""
    messages = []
    inventory = player_state.setdefault("inventory", {})
    for entry in drops:
        item_id = entry.get("id")
        qty = int(entry.get("quantity", 0))
        if not item_id or qty <= 0:
            continue
        if item_id == "blood_echoes":
            player_state["blood_echoes"] = int(player_state.get("blood_echoes", 0)) + qty
            messages.append(f"+{qty} blood echoes")
            continue
        inventory[item_id] = int(inventory.get(item_id, 0)) + qty
        item_data = GAME_DATA_ORIG.get(f"Items/{item_id}", {})
        item_name = item_data.get("name", item_id)
        messages.append(f"{item_name} x{qty}")
    return messages


# Attack action: enforce preconditions, distribute damage, handle drops/retaliation.
def Attack(object, instrument):
    ACTION_NAME = "attack"
    ACTION_DESCRIPTION = "Attack someone with a weapon"
    ACTION_ALIASES = ["hit", "stab", "shoot", "strike", "slash"]

    # Preconditions: must know where the player is and there must be hostiles.
    player_state = _load_player_state()
    current_loc = player_state.get("current_location_id")
    if not current_loc:
        print("You are nowhere. Cannot attack without a current location.")
        return None

    location_payload = _load_location(current_loc)
    room_chars = location_payload.get("characters", [])

    # Preconditions: instrument must resolve to a weapon the player owns.
    weapon_id = _resolve_item_id(str(instrument))
    if not weapon_id:
        print(f"Not a valid weapon")
        return None

    weapon_data = GAME_DATA_ORIG.get(f"Items/{weapon_id}", {})
    if weapon_data.get("category") != "weapon":
        print(f"The {weapon_data.get('name', weapon_id)} is not a weapon.")
        return None

    inventory = player_state.get("inventory", {})
    if int(inventory.get(weapon_id, 0)) <= 0:
        print(f"You do not have a {weapon_data.get('name', weapon_id)} equipped.")
        return None

    # Preconditions: object must name a hostile in the room; damage spreads to all hostiles.
    target_id = _resolve_character_id(str(object))
    if not target_id:
        print(f"Cannot find any target matching '{object}'.")
        return None

    def _is_hostile(char_payload: dict) -> bool:
        return not bool(char_payload.get("friendly", False))

    enemies_in_room = []
    target_present = False
    for entry in room_chars:
        char_id = entry.get("id")
        count = int(entry.get("count", 0))
        char_payload = GAME_DATA_ORIG.get(f"Characters/{char_id}", {})
        if count <= 0 or not char_payload:
            continue
        if not _is_hostile(char_payload):
            continue
        if char_id == target_id:
            target_present = True
        enemies_in_room.append((char_id, count, char_payload))

    if not target_present:
        print(f"{object} is not present to attack.")
        return None

    if not enemies_in_room:
        print("There is nothing hostile here to attack.")
        return None

    # Effects: compute per-target damage using weapon's stored damage only.
    weapon_damage = int(weapon_data.get("damage", 0))
    total_attack_damage = max(1, weapon_damage)

    attackable_instances = []
    for char_id, count, payload in enemies_in_room:
        health_list, vulnerable = _init_enemy_state(
            player_state,
            current_loc,
            char_id,
            count,
            int(payload.get("health", 1)),
            bool(payload.get("vulnerable", True)),
        )
        # Preconditions: bosses require stun item before they become vulnerable.
        if payload.get("stun_item_id") and not vulnerable:
            stun_name = GAME_DATA_ORIG.get(f"Items/{payload['stun_item_id']}", {}).get(
                "name", payload["stun_item_id"]
            )
            print(f"{payload.get('name', char_id)} shrugs off your blow. Use {stun_name} first.")
            return None
        living = [(idx, hp) for idx, hp in enumerate(health_list) if hp > 0]
        if not living:
            continue
        attackable_instances.append(
            {
                "id": char_id,
                "payload": payload,
                "health_list": health_list,
                "vulnerable": vulnerable,
                "living_indices": [idx for idx, _ in living],
            }
        )

    total_targets = sum(len(entry["living_indices"]) for entry in attackable_instances)
    if total_targets == 0:
        print("All selected targets are already down.")
        return None

    per_target_damage = max(1, math.floor(total_attack_damage / total_targets))
    defeat_messages = []
    drop_messages = []
    total_retaliation = 0
    boss_kills = 0

    for entry in attackable_instances:
        payload = entry["payload"]
        name = payload.get("name", entry["id"])
        for idx in entry["living_indices"]:
            entry["health_list"][idx] = max(0, entry["health_list"][idx] - per_target_damage)
            remaining = entry["health_list"][idx]
            if remaining <= 0:
                defeat_messages.append(f"{name} #{idx + 1} defeated.")
                drop_messages.extend(
                    _collect_drops(player_state, payload.get("drop_items", []))
                )
                if payload.get("stun_item_id"):
                    boss_kills += 1
            else:
                total_retaliation += int(payload.get("damage", 0))
        _update_enemy_state(
            player_state, current_loc, entry["id"], entry["health_list"], entry["vulnerable"]
        )

    # Prepare enemy health readout after the attack.
    enemy_health_lines = []
    for entry in attackable_instances:
        name = entry["payload"].get("name", entry["id"])
        hp_str = ", ".join(str(hp) for hp in entry["health_list"])
        enemy_health_lines.append(f"{name} HP: {hp_str}")

    # Update room characters based on survivors
    updated_chars = []
    for entry in room_chars:
        char_id = entry.get("id")
        original_count = int(entry.get("count", 0))
        if original_count <= 0:
            continue
        char_state = (
            player_state.get("character_state", {})
            .get(current_loc, {})
            .get(char_id, {})
            .get("health", [])
        )
        remaining = len([hp for hp in char_state if hp > 0]) or 0
        if remaining > 0:
            updated_chars.append({"id": char_id, "count": remaining})
    location_payload["characters"] = updated_chars

    # Enemies strike back
    player_health = int(player_state.get("health", 0))
    player_health = max(0, player_health - total_retaliation)
    player_state["health"] = player_health

    # Insight rewards
    if boss_kills:
        current_insight = int(player_state.get("insight", 0))
        max_insight = int(player_state.get("max_insight", 99))
        player_state["insight"] = min(max_insight, current_insight + boss_kills)

    _save_player_state(player_state)
    _save_location(current_loc, location_payload)

    # Reporting
    print(f"Attacking {object} with {instrument}")
    print(f"Dealt {per_target_damage} damage to {total_targets} target(s).")
    if defeat_messages:
        for msg in defeat_messages:
            print(msg)
    if enemy_health_lines:
        for line in enemy_health_lines:
            print(line)
    if drop_messages:
        print("Drops:", ", ".join(drop_messages))
        print("Added to your inventory.")
    if total_retaliation:
        print(f"Enemies retaliated for {total_retaliation} damage. Health now {player_health}.")
    else:
        print("No retaliation.")
    print(f"Insight: {player_state.get('insight', 0)}")
    return {
        "damage_per_target": per_target_damage,
        "targets_hit": total_targets,
        "defeated": defeat_messages,
        "enemy_health": enemy_health_lines,
        "drops": drop_messages,
        "retaliation_damage": total_retaliation,
        "player_health": player_health,
        "insight": player_state.get("insight", 0),
    }

## TALK ACTION

In [96]:
# Fetch a dialogue list for a character using its id to gets its dialogue_file.
def _load_dialogue(char_payload: dict) -> List[dict]:
    # Always resolve by character id in Dialogues/<id>.json
    stem = char_payload.get("id")
    if not stem:
        return []
    return GAME_DATA_ORIG.get(f"Dialogues/{stem}", [])


# Talk action: validate NPC presence/friendliness and emit the next available line.
def Talk(object):
    ACTION_NAME = "talk"
    ACTION_DESCRIPTION = "Talk to an NPC"
    ACTION_ALIASES = ["speak", "chat", "communicate"]

    player_state = _load_player_state()
    current_loc = player_state.get("current_location_id")
    if not current_loc:
        print("You are nowhere. Cannot talk without a current location.")
        return None

    target_id = _resolve_character_id(str(object))
    if not target_id:
        print(f"Cannot find anyone matching '{object}'.")
        return None

    location_payload = _load_location(current_loc)
    room_chars = location_payload.get("characters", [])

    # Ensure the target is present and friendly/alive.
    target_entry = next((c for c in room_chars if c.get("id") == target_id and c.get("count", 0) > 0), None)
    if not target_entry:
        print(f"{object} is not here to talk to.")
        return None

    char_payload = GAME_DATA_ORIG.get(f"Characters/{target_id}", {})
    if not char_payload:
        print(f"No data found for {object}.")
        return None
    if not char_payload.get("friendly", False):
        print(f"{char_payload.get('name', target_id)} is hostile and will not talk.")
        return None

    # Check if NPC is alive based on stored health in player_state.
    char_state = (
        player_state.get("character_state", {})
        .get(current_loc, {})
        .get(target_id, {})
        .get("health", [])
    )
    if char_state and all(hp <= 0 for hp in char_state):
        print(f"{char_payload.get('name', target_id)} cannot talk; they are down.")
        return None

    dialogue_entries = _load_dialogue(char_payload)
    if not dialogue_entries:
        print(f"{char_payload.get('name', target_id)} has nothing to say.")
        return None

    # Pick the next dialogue the player qualifies for by insight.
    insight = int(player_state.get("insight", 0))
    dialogue_state = player_state.setdefault("dialogue_state", {})
    last_seen_id = int(dialogue_state.get(target_id, 0)) #last seen id for this character dialogue

    eligible = [d for d in dialogue_entries if insight >= int(d.get("min_insight", 0))]
    eligible.sort(key=lambda d: int(d.get("id", 0)))
    next_line = next((d for d in eligible if int(d.get("id", 0)) > last_seen_id), None)

    if not next_line:
        # If nothing new, repeat the last eligible line.
        next_line = eligible[-1] if eligible else None
    if not next_line:
        print(f"{char_payload.get('name', target_id)} has nothing to say right now.")
        return None

    dialogue_state[target_id] = int(next_line.get("id", last_seen_id))
    _save_player_state(player_state)

    text = next_line.get("text", "")
    print(f"{char_payload.get('name', target_id)} says: {text}")
    return {"character": target_id, "dialogue_id": next_line.get("id"), "text": text} # Simply ignored and discarded if not used

## UPGRADE WEAPON ACTION

In [97]:

def Upgrade_Weapon(object):
    ACTION_NAME = "upgrade_weapon"
    ACTION_DESCRIPTION = "Upgrade a weapon using Blood Stone shards"
    ACTION_ALIASES = ["upgrade", "enhance", "fortify"]

    # Preconditions: weapon must resolve and be in inventory.
    weapon_id = _resolve_item_id(str(object))
    if not weapon_id:
        print(f"Cannot find any weapon matching '{object}'.")
        return None

    weapon_payload = GAME_DATA_ORIG.get(f"Items/{weapon_id}", {})
    if weapon_payload.get("category") != "weapon":
        print(f"{weapon_payload.get('name', weapon_id)} is not a weapon.")
        return None

    player_state = _load_player_state()
    inventory = player_state.setdefault("inventory", {})
    if int(inventory.get(weapon_id, 0)) <= 0:
        print(f"You do not have {weapon_payload.get('name', weapon_id)} in your inventory.")
        return None

    # Preconditions: must be at Hunter's Workshop.
    current_loc = player_state.get("current_location_id")
    if _normalize(str(current_loc)) != "hunters_workshop":
        print("You must be at the Hunter's Workshop to upgrade weapons.")
        return None

    # Determine current upgrade level for this weapon.
    weapon_levels = player_state.setdefault("weapon_levels", {})
    current_level = int(weapon_levels.get(weapon_id, 1))

    # Cost: 1 bloodstone shard per current level.
    shard_id = "bloodstone_shard"
    shard_cost = current_level
    shard_count = int(inventory.get(shard_id, 0))
    if shard_count < shard_cost:
        print(f"Not enough Bloodstone Shards. Need {shard_cost}, have {shard_count}.")
        return None

    # Effect: increase damage by 20%.
    base_damage = int(weapon_payload.get("damage", 0))
    new_damage = max(1, math.ceil(base_damage * 1.2))
    weapon_payload["damage"] = new_damage
    GAME_DATA_ORIG[f"Items/{weapon_id}"] = weapon_payload

    # Deduct shards and bump level.
    inventory[shard_id] = shard_count - shard_cost
    if inventory[shard_id] <= 0:
        inventory.pop(shard_id, None)
    weapon_levels[weapon_id] = current_level + 1
    _save_player_state(player_state)

    print(f"Upgraded {weapon_payload.get('name', weapon_id)} to level {current_level + 1}.")
    print(f"Damage increased to {new_damage}.")
    print(f"Spent {shard_cost} Bloodstone Shard(s). Remaining: {inventory.get(shard_id, 0)}.")
    return {
        "weapon_id": weapon_id,
        "new_level": current_level + 1,
        "new_damage": new_damage,
        "shards_spent": shard_cost,
        "shards_left": inventory.get(shard_id, 0),
    }

## INCREASE HEALTH ACTION

In [98]:

def Increase_Health():
    ACTION_NAME = "increase_health"
    ACTION_DESCRIPTION = "Increase player's maximum health"
    ACTION_ALIASES = ["boost_health", "enhance_vitality"]

    player_state = _load_player_state()
    current_loc = player_state.get("current_location_id")
    if _normalize(str(current_loc)) != "hunters_lawn":
        print("You must be at the Hunter's Lawn to increase health.")
        return None

    loc_payload = _load_location(current_loc)
    room_chars = loc_payload.get("characters", []) if loc_payload else []
    doll_present = any(c.get("id") == "plain_doll" and c.get("count", 0) > 0 for c in room_chars)
    if not doll_present:
        print("The Doll is not present; cannot increase health.")
        return None

    blood_echoes = int(player_state.get("blood_echoes", 0))
    cost = 100  # echoes per 5% increase
    if blood_echoes < cost:
        print(f"Not enough Blood Echoes. Need {cost}, have {blood_echoes}.")
        return None

    max_health = int(player_state.get("max_health", player_state.get("health", 0)))
    new_max = max(1, math.ceil(max_health * 1.05))
    player_state["max_health"] = new_max
    # Keep current health at or below new max.
    player_state["health"] = min(new_max, int(player_state.get("health", 0)))
    player_state["blood_echoes"] = blood_echoes - cost

    _save_player_state(player_state)

    print(f"Max health increased to {new_max}.")
    print(f"Blood Echoes spent: {cost}. Remaining: {player_state['blood_echoes']}.")
    return {
        "max_health": new_max,
        "health": player_state["health"],
        "blood_echoes_spent": cost,
        "blood_echoes_left": player_state["blood_echoes"],
    }


## HEAL ACTION

In [99]:

def Heal():
    ACTION_NAME = "heal"
    ACTION_DESCRIPTION = "Heal yourself"
    ACTION_ALIASES = ["restore", "cure"]

    player_state = _load_player_state()
    inventory = player_state.setdefault("inventory", {})

    # Preconditions: player must be alive.
    if int(player_state.get("health", 0)) <= 0:
        print("You are down and cannot heal.")
        return None

    # Preconditions: must have a blood vial.
    if int(inventory.get("blood_vial", 0)) <= 0:
        print("No blood vials available to heal.")
        return None

    heal_amount = 100
    max_hp = int(player_state.get("max_health", player_state.get("health", 0)))
    player_state["health"] = min(max_hp, int(player_state.get("health", 0)) + heal_amount)

    # Consume one blood vial.
    inventory["blood_vial"] = int(inventory.get("blood_vial", 0)) - 1
    if inventory["blood_vial"] <= 0:
        inventory.pop("blood_vial", None)

    _save_player_state(player_state)

    print(f"Healed {heal_amount}. Health is now {player_state['health']}.")
    return {"healed": heal_amount, "health": player_state["health"]}

## USE ACTION

In [100]:
def Use(object):
    ACTION_NAME = "use"
    ACTION_DESCRIPTION = "Use an item from inventory"
    ACTION_ALIASES = ["consume", "activate"]

    player_state = _load_player_state()
    inventory = player_state.setdefault("inventory", {})
    current_loc = player_state.get("current_location_id")

    item_id = _resolve_item_id(str(object))
    if not item_id:
        print(f"Cannot find any item matching '{object}'.")
        return None

    item_payload = GAME_DATA_ORIG.get(f"Items/{item_id}", {})

    # Lamps: check presence in the room, not inventory.
    if item_id == "lamp":
        # Lamp check: presence in the current room items, not inventory.
        location_payload = _load_location(current_loc) if current_loc else {}
        room_items = location_payload.get("items", []) if location_payload else []
        lamp_present = any(i.get("id") == "lamp" and i.get("count", 0) > 0 for i in room_items)
        if not lamp_present:
            print("There is no lamp here to use.")
            return None

        unlocked_map = player_state.get("unlocked_lamps", {}) or {}
        unlocked = [loc_id for loc_id, val in unlocked_map.items() if val]
        if not unlocked:
            print("No lamps unlocked to teleport to.")
            return None
        print("Where do you wanna Teleport to?")
        for idx, lamp in enumerate(unlocked, 1):
            print(f"{idx}. {lamp}")
        choice = input("Enter a number or location id: ").strip()
        destination = None
        if choice.isdigit():
            num = int(choice)
            if 1 <= num <= len(unlocked):
                destination = unlocked[num - 1]
        else:
            for lamp in unlocked:
                if _normalize(lamp) == _normalize(choice):
                    destination = lamp
                    break
        if not destination:
            print("Invalid selection. No teleport performed.")
            return None
        Teleport(destination)
        return {"used": item_id, "options": unlocked, "teleported_to": destination}

    else:
        if int(inventory.get(item_id, 0)) <= 0:
            print(f"You do not have {item_payload.get('name', item_id)} in your inventory.")
            return None

    # Special cases by item id/category
    if item_id == "blood_echoes":
        if _normalize(str(current_loc)) != "hunters_lawn":
            print("Blood Echoes can only be used at the Hunter's Lawn with the Doll present.")
            return None
        result = Increase_Health()
        return {"used": item_id, "result": result}

    if item_id == "bloodstone_shard":
        # Must be at Hunter's Workshop to upgrade.
        if _normalize(str(current_loc)) != "hunters_workshop":
            print("You must be at the Hunter's Workshop to use Bloodstone Shards.")
            return None
        # Then it will map to bloodstone_shard
        print("What weapon do you want to upgrade?")
        return {"used": item_id}



    # Stun items: mark bosses/hostiles in room as vulnerable if matching stun_item_id.
    location_payload = _load_location(current_loc) if current_loc else {}
    room_chars = location_payload.get("characters", []) if location_payload else []
    stunned_any = False
    for entry in room_chars:
        char_id = entry.get("id")
        char_payload = GAME_DATA_ORIG.get(f"Characters/{char_id}", {})
        if not char_payload:
            continue
        if char_payload.get("stun_item_id") == item_id:
            # Flip vulnerable to true in character_state for this location/character.
            char_state = player_state.setdefault("character_state", {}).setdefault(current_loc, {})
            state_entry = char_state.setdefault(char_id, {"health": [char_payload.get("health", 1)], "vulnerable": False})
            state_entry["vulnerable"] = True
            stunned_any = True
            print(f"{char_payload.get('name', char_id)} is now vulnerable.")
    if stunned_any:
        _save_player_state(player_state)
        return {"used": item_id, "stunned": True}

    # Generic consumable: blood_vial heals via Heal(), others just consume.
    if item_id == "blood_vial":
        result = Heal()
        _save_player_state(player_state)
        return {"used": item_id, "result": result}

    # Default: consume one and print.
    inventory[item_id] -= 1
    if inventory[item_id] <= 0:
        inventory.pop(item_id, None)
    _save_player_state(player_state)
    print(f"Used {item_payload.get('name', item_id)}.")
    return {"used": item_id}


## GET ACTION

In [101]:
def Get(object):
    ACTION_NAME = "get"
    ACTION_DESCRIPTION = "Pick up an item"
    ACTION_ALIASES = ["take", "grab", "pickup", "collect"]

    player_state = _load_player_state()
    current_loc = player_state.get("current_location_id")
    if not current_loc:
        print("You are nowhere. Cannot get items without a current location.")
        return None

    item_id = _resolve_item_id(str(object))
    if not item_id:
        print(f"Cannot find any item matching '{object}'.")
        return None

    item_payload = GAME_DATA_ORIG.get(f"Items/{item_id}", {})

    # Check if item can be picked up
    if not item_payload.get("gettable", False):
        item_name = item_payload.get('name', item_id)
        print(f"{item_name} cannot be picked up.")
        return None

    location_payload = _load_location(current_loc)
    items = location_payload.get("items", [])
    found = next((i for i in items if i.get("id") == item_id and i.get("count", 0) > 0), None)

    if not found:
        print(f"{item_payload.get('name', item_id)} is not present here.")
        return None

    available = int(found.get("count", 0))
    if available <= 0:
        print(f"{item_payload.get('name', item_id)} is not present here.")
        return None

    # Remove all of this item from the location
    items.remove(found)
    location_payload["items"] = items

    # Add the full stack to inventory (handle int or dict schemas)
    inventory = player_state.setdefault("inventory", {})
    inv_entry = inventory.get(item_id)
    if isinstance(inv_entry, dict):
        inv_entry["quantity"] = int(inv_entry.get("quantity", 0)) + available
        inventory[item_id] = inv_entry
        new_total = inv_entry["quantity"]
    else:
        inventory[item_id] = int(inv_entry or 0) + available
        new_total = inventory[item_id]

    _save_location(current_loc, location_payload)
    _save_player_state(player_state)

    item_name = item_payload.get('name', item_id)
    print(f"Picked up {available} x {item_name}.")
    return {"picked_up": item_id, "quantity": available, "inventory_count": new_total}


## TRAVEL ACTION

In [102]:
import re

def normalize_name(name: str) -> str:
    """
    Normalize a name for fuzzy matching:
    - Convert to lowercase
    - Remove apostrophes
    - Remove extra whitespace
    - Remove special characters
    """
    if not name:
        return ""
    name = name.lower()
    name = name.replace("'", "")  # Remove apostrophes
    name = name.replace("'", "")  # Remove smart apostrophes
    name = re.sub(r'[^\w\s]', '', name)  # Remove special chars except spaces
    name = re.sub(r'\s+', ' ', name)  # Normalize whitespace
    return name.strip()


def get_location_by_name_fuzzy(target_name: str):
    """
    Find a location by name with fuzzy matching.
    """
    if not target_name:
        return None

    # Try exact match first
    location = get_location_by_name(target_name)
    if location:
        return location

    # Try normalized fuzzy match
    normalized_target = normalize_name(target_name)

    # Search through all locations in GAME_DATA_ORIG
    for key, loc_data in GAME_DATA_ORIG.items():
        if not key.startswith("Locations/"):
            continue

        loc_name = loc_data.get("name", "")
        loc_id = loc_data.get("id", "")

        normalized_loc_name = normalize_name(loc_name)
        normalized_loc_id = normalize_name(loc_id)

        # Check if normalized names match
        if normalized_loc_name == normalized_target or normalized_loc_id == normalized_target:
            return loc_data

    return None



def Travel(object: str):
    destination_name = object.strip()

    player_state = _load_player_state()
    current_location_id = player_state["current_location_id"]
    current_location_data = get_location_by_id(current_location_id)

    if not current_location_data:
        print(f"[ERROR] Current location '{current_location_id}' not found in game data.")
        return

    # Try to find destination
    destination_data = get_location_by_id(destination_name)
    if not destination_data:
        destination_data = get_location_by_name_fuzzy(destination_name)

    if not destination_data:
        print(f"[ERROR] Location '{destination_name}' does not exist.")
        return

    # Check if already there
    if destination_data["id"] == current_location_id:
        print(f"[INFO] You are already in '{destination_name}'.")
        Describe_Room()
        return

    # Check if adjacent
    is_adjacent = False
    if current_location_data.get("previous") == destination_data["id"]:
        is_adjacent = True
    elif current_location_data.get("next") == destination_data["id"]:
        is_adjacent = True

    if not is_adjacent:
        print(
            f"[ERROR] You cannot travel directly to '{destination_name}' "
            f"from '{current_location_data['name']}'. It's not an adjacent exit."
        )
        return

    # Key item check
    key_item = destination_data.get("key_item_for_travel")
    if key_item and key_item not in [None, "null", ""]:
        inv = player_state.get("inventory", {})
        # Handle both dict and int inventory formats
        has_key = False
        if key_item in inv:
            if isinstance(inv[key_item], dict):
                has_key = inv[key_item].get("quantity", 0) > 0
            else:
                has_key = int(inv[key_item]) > 0

        if not has_key:
            print(f"You need the '{key_item}' to enter '{destination_data['name']}'.")
            return

    # Move player
    player_state["current_location_id"] = destination_data["id"]
    _save_player_state(player_state)

    print(f"Traveled to {destination_data['name']}.")

    # Handle visited locations and lamps
    visited = player_state.setdefault("visited_locations", {})
    lamps = player_state.setdefault("unlocked_lamps", {})

    if not visited.get(destination_data["id"], False):
        visited[destination_data["id"]] = True
        if location_has_lamp(destination_data) and not lamps.get(destination_data["id"], False):
            lamps[destination_data["id"]] = True
            player_state["insight"] = player_state.get("insight", 0) + 1
            _save_player_state(player_state)
            print(
                f"You've lit a new lamp in {destination_data['name']} "
                f"and increased your insight to {player_state['insight']}."
            )

    Describe_Room()



## TELEPORT ACTION

In [120]:
def Teleport(object):

    destination_name = object.strip()
    if not destination_name:
        print("[ERROR] Teleport where? (e.g., 'teleport hunter's lawn')")
        return
    player_state = _load_player_state()
    current_location_id = player_state["current_location_id"]
    current_location_data = get_location_by_id(current_location_id)

    # Resolve destination: try id first, then name
    destination_data = get_location_by_id(destination_name)
    if not destination_data:
        destination_data = get_location_by_name(destination_name)

    if not destination_data:
        print(f"[ERROR] Location '{destination_name}' does not exist.")
        return

    dest_id = destination_data["id"]
    dest_name = destination_data.get("name", dest_id)

    # Ensure dicts exist even if missing in save / initial state
    lamps   = player_state.setdefault("unlocked_lamps", {})
    visited = player_state.setdefault("visited_locations", {})


    # 1. Must have a lit lamp there
    if not lamps.get(dest_id, False):
        print(f"You cannot teleport to {dest_name} — its lamp has not been lit yet.")
        return

    # 3. Must not be current location
    if dest_id == current_location_id:
        print(f"[INFO] You are already at {dest_name}.")
        Describe_Room()
        return

    # Key item check
    key_item = destination_data.get("key_item_for_travel")
    if key_item and key_item not in [None, "null", ""]:
        inv = player_state.get("inventory", {})
        # Handle both dict and int inventory formats
        has_key = False
        if key_item in inv:
            if isinstance(inv[key_item], dict):
                has_key = inv[key_item].get("quantity", 0) > 0
            else:
                has_key = int(inv[key_item]) > 0

        if not has_key:
            print(f" You need the '{key_item}' to enter '{destination_data['name']}'.")
            return

    # ---- Effects ----
    # 1. Move player
    PLAYER_STATE["current_location_id"] = dest_id
    print(f"You focus on the lamp's pale flame and teleport to {dest_name}.")

    # 2. If somehow this is the *first* time counted as visited, mark + insight
    if not visited.get(dest_id, False):
        visited[dest_id] = True

    # 3. Describe room
    Describe_Room()

## INSPECT ACTION

In [112]:
def Inspect(object):
    ACTION_NAME = "inspect"
    ACTION_DESCRIPTION = "Examine an item or character for details"
    ACTION_ALIASES = ["examine", "look", "check", "read"]

    print(f"Inspecting {object}")
    target_text = object.strip()
    if not target_text:
        print("[ERROR] Inspect what?")
        return

    # Use attack helpers to resolve ids
    item_id = _resolve_item_id(target_text)
    char_id = _resolve_character_id(target_text)

    # Load state/location via helpers so we're consistent with Attack()
    player_state = _load_player_state() or PLAYER_STATE
    current_loc = player_state.get("current_location_id") or PLAYER_STATE["current_location_id"]
    loc_payload = _load_location(current_loc)

    inventory = player_state.get("inventory", {})
    room_items = loc_payload.get("items", [])
    room_characters = loc_payload.get("characters", [])

    # ------------------------------------------------
    # 1) ITEM INSPECT (inventory or room)
    # ------------------------------------------------
    if item_id:
        in_inventory = item_id in inventory
        in_room = any(entry.get("id") == item_id for entry in room_items)

        if not in_inventory and not in_room:
            print(f"[ERROR] You don't see any '{object}' here or in your inventory.")
            return

        item_data = GAME_DATA_ORIG.get(f"Items/{item_id}", {}) or {}
        name = item_data.get("name", item_id)
        desc = item_data.get("description", "No description available.")
        category = item_data.get("category", "unknown").lower()

        print(f"\n=== {name} ===")
        print(desc)

        print("\nDetails:")
        print(f" - Category: {category}")

        if in_inventory:
            # Your inventory schema can be dict of dicts or ints; handle both
            inv_entry = inventory.get(item_id)
            if isinstance(inv_entry, dict):
                qty = inv_entry.get("quantity", 1)
                level = inv_entry.get("level")
            else:
                qty = int(inv_entry)
                level = None
            print(f" - Quantity (owned): {qty}")

            if level is not None:
                print(f" - Upgrade Level: {level}")

        if in_room:
            room_entry = next(e for e in room_items if e.get("id") == item_id)
            print(f" - Quantity here: {room_entry.get('count', 1)}")

        # Category-specific info
        if category == "weapon":
            print(f" - Damage: {item_data.get('damage', '?')}")

        elif category == "consumable":
            print(f" - Healing: {item_data.get('healing', '?')} HP")
            print(f" - Used for: {item_data.get('usable_for', 'unknown')}")

        elif category == "key_item":
            gate = item_data.get("gate_unlocked")
            if gate:
                print(f" - Unlocks: {gate}")

        print()
        return

    # ------------------------------------------------
    # 2) CHARACTER INSPECT (must be in current room)
    # ------------------------------------------------
    if char_id:
        if not any(entry.get("id") == char_id for entry in room_characters):
            print(f"[ERROR] {object} is not here.")
            return

        char_data = GAME_DATA_ORIG.get(f"Characters/{char_id}", {}) or {}
        name = char_data.get("name", char_id)
        desc = char_data.get("description", "No description available.")

        # Friendly vs hostile (Attack uses 'friendly' and/or 'is_hostile')
        friendly = char_data.get("friendly")
        if friendly is None:
            friendly = not bool(char_data.get("is_hostile", False))

        health = char_data.get("health", "?")
        damage = char_data.get("damage", "?")
        vulnerable = char_data.get("vulnerable", False)

        # How many copies are in the room
        room_entry = next(e for e in room_characters if e.get("id") == char_id)
        count = int(room_entry.get("count", 1))

        print(f"\n=== {name} ===")
        print(desc)

        print("\nDetails:")
        print(f" - Disposition: {'Friendly' if friendly else 'Not friendly'}")
        print(f" - Health: {health}")
        print(f" - Damage: {damage}")
        print(f" - Vulnerable: {vulnerable}")
        if count > 1:
            print(f" - Number present: {count}")

        print()
        return

    # ------------------------------------------------
    # 3) Nothing matched
    # ------------------------------------------------
    print(f"[ERROR] There is no '{object}' here or in your inventory.")


## DESCRIBE ROOM

In [113]:
def Describe_Room():
    ACTION_NAME = "describe_room"
    ACTION_DESCRIPTION = "Describe the current room/location"
    ACTION_ALIASES = ["look around", "examine room", "room description"]

    player_state = _load_player_state()
    current_location = player_state["current_location_id"]
    current_location_data = get_location_by_id(current_location)

    if not current_location_data:
        print(f"[ERROR] Current location '{current_location}' not found in game data.")
        return

    name = current_location_data.get("name", current_location)
    description = current_location_data.get("description", "There is nothing notable here.")

    print(f"\n=== {name} === // {current_location}")
    print(description)

    items = current_location_data.get("items", [])
    if items:
        print("\nItems here:")
        for entry in items:
            item_id = entry.get("id")
            qty = entry.get("count", 1)
            item_payload = GAME_DATA_ORIG.get(f"Items/{item_id}", {})
            item_name = item_payload.get("name", item_id.replace("_", " ").title())
            print(f" - {item_name} (x{qty})")
    else:
        print("\nNo items are visible.")

    characters = current_location_data.get("characters", [])
    if characters:
        print("\nYou see:")
        for ch in characters:
            ch_id = ch.get("id", "unknown_character")
            ch_payload = GAME_DATA_ORIG.get(f"Characters/{ch_id}", {})
            ch_name = ch_payload.get("name", ch_id.replace("_", " ").title())
            qty = ch.get("count", 1)
            if qty > 1:
                print(f" - {qty} × {ch_name}")
            else:
                print(f" - {ch_name}")
    else:
        print("\nNo one else is here.")

    prev_exit = current_location_data.get("previous")
    next_exit = current_location_data.get("next")

    print("\nExits:")
    if prev_exit:
        prev_loc = get_location_by_id(prev_exit)
        prev_name = prev_loc["name"] if prev_loc else prev_exit
        print(f" - Previous: {prev_name} ({prev_exit})")
    if next_exit:
        next_loc = get_location_by_id(next_exit)
        next_name = next_loc["name"] if next_loc else next_exit
        print(f" - Next: {next_name} ({next_exit})")

    if not prev_exit and not next_exit:
        print(" - None")

    print("\n")

## INVENTORY ACTION

In [114]:
def Inventory_Stats():
    ACTION_NAME = "inventory_stats"
    ACTION_DESCRIPTION = "Display player's inventory statistics"
    ACTION_ALIASES = ["inventory stats", "show inventory", "display inventory", "inventory"]

    player_state = _load_player_state()
    inventory = player_state.get("inventory", {})

    if not inventory:
        print("Your inventory is empty.")
        return

    print("\n===== Player's Inventory =====")

    for item_id, inv_data in inventory.items():
        # Handle both dict format and int format
        if isinstance(inv_data, dict):
            qty = inv_data.get("quantity", 0)
            level = inv_data.get("level")
        else:
            qty = int(inv_data)
            level = None

        # Get item definition from GAME_DATA_ORIG
        item_def = GAME_DATA_ORIG.get(f"Items/{item_id}", {})

        if item_def:
            name = item_def.get("name", item_id)
            category = item_def.get("category", "misc")
            description = item_def.get("description", "")
            damage = item_def.get("damage")
            healing = item_def.get("healing")
        else:
            name = item_id.replace("_", " ").title()
            category = "unknown"
            description = ""
            damage = None
            healing = None

        # Build display line
        line = f"- {name} (x{qty}) [{category}]"

        if category == "weapon":
            if level is not None:
                line += f" — Weapon Lv {level}"
            if damage is not None:
                line += f" — Damage {damage}"
        elif category == "consumable":
            if healing is not None:
                line += f" — Heals {healing} HP"

        print(line)

        if description:
            print(f"    {description}")

    print("================================\n")

## RESET GAME

In [115]:
def reset_game():
    """
    Reload all game data from JSON files and reset player state.
    Use this when you've updated the JSON files and want a fresh start.
    """
    global GAME_DATA_ORIG, PLAYER_STATE, LOCATION_INDEX

    print("🔄 Resetting game...")

    # CRITICAL: Clear all references first
    GAME_DATA_ORIG = None
    PLAYER_STATE = None
    LOCATION_INDEX = None

    # Reload all JSON data from disk (this gets the updated GitHub files)
    GAME_DATA_ORIG = load_json_tree(ROOT)

    # Rebuild location index
    LOCATION_INDEX = {}
    for key, data in GAME_DATA_ORIG.items():
        if key.startswith("Locations/") and isinstance(data, dict) and "id" in data:
            LOCATION_INDEX[data["id"]] = data

    # CRITICAL: Make a DEEP COPY of the fresh player_state from the reloaded JSON
    fresh_player_state = GAME_DATA_ORIG.get("player_state", {})
    PLAYER_STATE = copy.deepcopy(fresh_player_state)

    # Update GAME_DATA_ORIG to use this fresh copy
    GAME_DATA_ORIG["player_state"] = PLAYER_STATE

    # Verify the inventory is empty
    inventory = PLAYER_STATE.get("inventory", {})

    print("\n Game reset complete! Ready to play.\n")

## PLAYER STATS ACTION

In [116]:

def Player_Stats():
    ACTION_NAME = "player_stats"
    ACTION_DESCRIPTION = "Display player's current statistics"
    ACTION_ALIASES = ["show stats", "display stats", "stats"]

    state = _load_player_state()
    health = int(state.get("health", 0))
    if health <= 0:
        print("Player is down. No stats to display.")
        return None

    max_health = int(state.get("max_health", health))
    location = state.get("current_location_id", "unknown")
    insight = int(state.get("insight", 0))
    blood_echoes = int(state.get("blood_echoes", 0))
    inventory = state.get("inventory", {})
    blood_shards = int(inventory.get("bloodstone_shard", 0))

    print(f"Health: {health}/{max_health}")
    print(f"Location: {location}")
    print(f"Insight: {insight}")
    print(f"Blood Echoes: {blood_echoes}")
    print(f"Bloodstone Shards: {blood_shards}")

    return {
        "health": health,
        "max_health": max_health,
        "location": location,
        "insight": insight,
        "blood_echoes": blood_echoes,
        "bloodstone_shards": blood_shards,
    }

In [117]:
# ----------------------------
# Dispatcher
# ----------------------------
ACTION_DISPATCH = {
    "attack": Attack,
    "talk": Talk,
    "use": Use,
    "upgrade_weapon": Upgrade_Weapon,
    "travel": Travel,
    "teleport": Teleport,
    "get": Get,
    "inspect": Inspect,
    "heal": lambda obj=None, inst=None: Heal(),
    "increase_health": lambda obj=None, inst=None: Increase_Health(),
    "inventory_stats": lambda obj=None, inst=None: Inventory_Stats(),
    "player_stats": lambda obj=None, inst=None: Player_Stats(),
    "describe_room": lambda obj=None, inst=None: Describe_Room(),
    "reset_game": lambda obj=None, inst=None: reset_game(),
}

## GAME LOOP

In [118]:
def _gehrman_health(state: dict) -> int:
    """
    Return Gehrman's current health if tracked in state; otherwise fall back to base data.
    Gehrman is stored as a character JSON, so base health is available via GAME_DATA_ORIG.
    """
    char_state = state.get("character_state", {})
    for _, entries in char_state.items():
        gehr = entries.get("gehrman", {})
        health_list = gehr.get("health", [])
        if health_list:
            return sum(health_list)
    base = GAME_DATA_ORIG.get("Characters/gehrman", {})
    return int(base.get("health", 0))


def main_loop():
    print("Bloodborne: Text Adventure Game")
    print("==================================")
    Describe_Room()

    while True:
        state = _load_player_state()
        player_health = int(state.get("health", 0))
        gehr_health = _gehrman_health(state)

        if player_health <= 0:
            print("Lost, the hunter died.")
            break
        if gehr_health <= 0:
            print("Won, the night is over.")
            break

        user_input = input("\n>>> ").strip()
        if user_input.lower() in ["quit", "exit"]:
            break
        if not user_input:
            continue
        run_command(user_input)


In [122]:
reset_game()
main_loop()

🔄 Resetting game...

 Game reset complete! Ready to play.

Bloodborne: Text Adventure Game

=== Hunter's Lawn === // hunters_lawn
A peaceful garden illuminated by a mystical lamp. A gentle doll watches silently over the grounds.

Items here:
 - Hunter's Lamp (x1)
 - Hunter Axe (x1)

You see:
 - Plain Doll

Exits:
 - Next: Hunter's Workshop (hunters_workshop)



>>> pick up hunter axe
Picked up Hunter Axe.

>>> use lamp
Where do you wanna Teleport to?
1. hunters_lawn
2. operation_theater
Enter a number or location id: 2
You focus on the lamp's pale flame and teleport to Operation Theater.

=== Operation Theater === // operation_theater
A dimly lit surgery room filled with mysterious medical instruments. A calm woman observes you carefully.

Items here:
 - Note (x1)
 - Hunter's Lamp (x1)

You see:
 - Iosefka

Exits:
 - Next: Sick Room (sick_room)



>>> inspect note
Inspecting note

=== Note ===
A hastily scrawled message: "seek paleblood to transcend the hunt".

Details:
 - Category: ke